In [6]:
!pip install openai fastmcp

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 29.0 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.9.0-py3-none-any.whl size=11103 sha256=169534c63b1993ca775f6d89585f6c34c78e3aec849438646f2d88d242246814
  Stored in directory: /home/codespace/.cache/pip/wheels/e0/e8/fc/8ab8aa326e33bc066ccd5f3ca9646eab4299881af933f94f09
Successfully built pyperclip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16/16 [fastmcp]5;237m━━ 15/16 [fastmcp]n]


In [7]:
import os

In [16]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    """Retrieves the temperature for a specified city."""
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

get_weather_tool = {
    "type": "function",
    "name": "get_weather", # TODO1: The function's name
    "description": "Retrieves the temperature for a specified city.", # TODO2: A description for the LLM
    "parameters": {
        "type": "object",
        "properties": {
            "city": { # TODO3: The function's parameter name
                "type": "string",
                "description": "The name of the city for which to retrieve weather data." # TODO4: A description of the parameter
            }
        },
        "required": ["city"], # TODO5: A list of required parameters
        "additionalProperties": False
    }
}

todo3 = get_weather_tool['parameters']['properties'].keys()
todo3

dict_keys(['city'])

In [17]:
# Define the set_weather function
def set_weather(city: str, temp: float) -> str:
    """Sets the temperature for a specified city."""
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Sets the temperature for a specified city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city for which to set the weather data."
            },
            "temp": {
                "type": "number",
                "description": "The temperature to associate with the city."
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

import json
print(json.dumps(set_weather_tool, indent=4))

{
    "type": "function",
    "name": "set_weather",
    "description": "Sets the temperature for a specified city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city for which to set the weather data."
            },
            "temp": {
                "type": "number",
                "description": "The temperature to associate with the city."
            }
        },
        "required": [
            "city",
            "temp"
        ],
        "additionalProperties": false
    }
}


In [10]:
!pip show fastmcp

Name: fastmcp
Version: 2.10.5
Summary: The fast, Pythonic way to build MCP servers and clients.
Home-page: https://gofastmcp.com
Author: Jeremiah Lowin
Author-email: 
License-Expression: Apache-2.0
Location: /usr/local/python/3.12.1/lib/python3.12/site-packages
Requires: authlib, cyclopts, exceptiongroup, httpx, mcp, openapi-pydantic, pydantic, pyperclip, python-dotenv, rich
Required-by: 


In [11]:
%%writefile weather_server.py

from fastmcp import FastMCP
import random

known_weather_data = {'berlin': 20.0}

mcp = FastMCP("Weather Service ☁️")

@mcp.tool
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.
    """
    city = city.strip().lower()
    if city in known_weather_data:
        return known_weather_data[city]
    return round(random.uniform(-5, 35), 1)

@mcp.tool
def set_weather(city: str, temp: float) -> str:
    """
    Sets the temperature for a specified city.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

if __name__ == "__main__":
    mcp.run()

Writing weather_server.py


In [18]:
import mcp_client

our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

available_tools = our_mcp_client.get_tools()

our_mcp_client.stop_server()

# print(json.dumps(available_tools, indent=4))

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Weather Service ☁️', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully
Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Server stopped
